In [2]:
import os
os.environ["HF_DATASETS_OFFLINE"] = "1"
from datasets import load_dataset
single_ds = load_dataset("NousResearch/hermes-function-calling-v1", "func_calling_singleturn")
multi_ds = load_dataset("NousResearch/hermes-function-calling-v1", "func_calling")
glaive_ds = load_dataset("NousResearch/hermes-function-calling-v1", "glaive_func_calling")

/Users/admin/fish/octupus-tool-call/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since NousResearch/hermes-function-calling-v1 couldn't be found on the Hugging Face Hub (offline mode is enabled).
Found the latest cached dataset configuration 'func_calling_singleturn' at /Users/admin/.cache/huggingface/datasets/NousResearch___hermes-function-calling-v1/func_calling_singleturn/0.0.0/8f025148382537ba84cd325e1834b706e1461692 (last modified on Sun Dec 22 19:10:52 2024).
Using the latest cached version of the dataset since NousResearch/hermes-function-calling-v1 couldn't be found on the Hugging Face Hub (offline mode is enabled).
Found the latest cached dataset configuration 'func_calling' at /Users/admin/.cache/huggingface/datasets/NousResearch___hermes-

In [3]:
glaive_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'conversations', 'tools', 'category', 'subcategory', 'task', 'source'],
        num_rows: 5209
    })
})

In [4]:
import json
import re
from pprint import pprint
from copy import deepcopy

def map_func(example):
    example = deepcopy(example)
    ret = {}
    ret["conversation"] = example["conversations"]

    messages = []
    if example["id"] == "e39781a4-29a3-4896-83d2-0ab5d264b6ed":
        ret["conversation"] = messages
        ret["tools"] = None
        return ret
    try:
        for c in ret["conversation"]:
            if c["from"] == "system":
                tools = re.search(r'<tools>\n(.*?)\n</tools>', c["value"], re.DOTALL).group(1)
                ret["tools"] = json.dumps([t for t in eval(tools)], ensure_ascii=False)
            elif c["from"] == "human":
                messages.append({"role": "user", "content": c["value"]})
            elif c["from"] == "gpt":
                tool_calls = re.findall(r'<tool_call>\n(.*?)\n</tool_call>', c["value"], re.DOTALL)
                if tool_calls:
                    tool_calls = [{"type": "function", "function": {"name": eval(t)["name"], "arguments": json.dumps(eval(t)["arguments"], ensure_ascii=False)}} for t in tool_calls]
                    content = re.search(r'^(.*?)<tool_call>\n', c["value"], re.DOTALL).group(1)
                else:
                    content = c["value"]
                    tool_calls = None
                messages.append({"role": "assistant", "content": content, "tool_calls": tool_calls})
            elif c["from"] == "tool":
                tool_rsps = re.findall(r'<tool_response>\n(.*?)\n</tool_response>', c["value"], re.DOTALL)
                for t in tool_rsps:
                    messages.append({"role": "tool", "content": json.dumps(eval(t), ensure_ascii=False)})
    except:
        messages = []
        ret["tools"] = None
    ret["conversation"] = messages
    return ret


def glaive_map_func(example):
    example = deepcopy(example)
    ret = {}
    ret["conversation"] = example["conversations"]

    messages = []
    try:
        for c in ret["conversation"]:
            if c["from"] == "human":
                messages.append({"role": "user", "content": c["value"]})
            elif c["from"] == "gpt":
                tool_calls = re.findall(r'<tool_call>\n(.*?)\n</tool_call>', c["value"], re.DOTALL)
                if tool_calls:
                    tool_calls = [{"type": "function", "function": {"name": eval(t)["name"], "arguments": json.dumps(eval(t)["arguments"], ensure_ascii=False)}} for t in tool_calls]
                    content = re.search(r'^(.*?)<tool_call>\n', c["value"], re.DOTALL).group(1)
                else:
                    content = c["value"]
                    tool_calls = None
                messages.append({"role": "assistant", "content": content, "tool_calls": tool_calls})
            elif c["from"] == "tool":
                tool_rsps = re.findall(r'<tool_response>\n(.*?)\n</tool_response>', c["value"], re.DOTALL)
                for t in tool_rsps:
                    messages.append({"role": "tool", "content": json.dumps(eval(t), ensure_ascii=False)})
    except:
        messages = []
    if json.loads(example["tools"]):
        try:
            ret["tools"] = json.dumps([t for t in json.loads(example["tools"])], ensure_ascii=False)
        except:
            messages = []
            ret["tools"] = None
    else:
        ret["tools"] = None
    ret["conversation"] = messages
    ret["subcategory"] = ""
    return ret

In [5]:
ds1 = single_ds["train"].map(map_func, remove_columns=["conversations"]).filter(lambda x: x["conversation"] and x["tools"])
ds2 = multi_ds["train"].map(map_func, remove_columns=["conversations"]).filter(lambda x: x["conversation"] and x["tools"])
ds3 = glaive_ds["train"].map(glaive_map_func, remove_columns=["conversations", 'source']).filter(lambda x: x["conversation"] and x["tools"])

from datasets import DatasetDict

all_ds = DatasetDict({
    "func_calling_singleturn": ds1,
    "func_calling": ds2,
    "glaive_func_calling": ds3
})
all_ds

DatasetDict({
    func_calling_singleturn: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'tools'],
        num_rows: 1832
    })
    func_calling: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'tools'],
        num_rows: 1653
    })
    glaive_func_calling: Dataset({
        features: ['id', 'tools', 'category', 'subcategory', 'task', 'conversation'],
        num_rows: 4344
    })
})

In [7]:
import checker_oai
import importlib
importlib.reload(checker_oai)
ck = checker_oai.Checker()

all_ds = all_ds.filter(lambda x: ck.check(x) is None, load_from_cache_file=False)
all_ds

Filter:   0%|          | 0/1832 [00:00<?, ? examples/s]ERROR:root:True is not one of [None]

Failed validating 'enum' in schema['properties']['pretrained']:
    {'type': 'boolean',
     'description': 'Flag to indicate if the model should be loaded with '
                    'pre-trained weights.',
     'enum': [None]}

On instance['pretrained']:
    True
Traceback (most recent call last):
  File "/Users/admin/fish/octupus-tool-call/data/checker_oai.py", line 75, in check
    self.check_schema(data)
  File "/Users/admin/fish/octupus-tool-call/data/checker_oai.py", line 59, in check_schema
    raise e
  File "/Users/admin/fish/octupus-tool-call/data/checker_oai.py", line 54, in check_schema
    validate(instance=arguments, schema=schema)
  File "/Users/admin/fish/octupus-tool-call/venv/lib/python3.11/site-packages/jsonschema/validators.py", line 1332, in validate
    raise error
jsonschema.exceptions.ValidationError: True is not one of [None]

Failed validating 'enum' in schema['propert

DatasetDict({
    func_calling_singleturn: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'tools'],
        num_rows: 1783
    })
    func_calling: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'tools'],
        num_rows: 1604
    })
    glaive_func_calling: Dataset({
        features: ['id', 'tools', 'category', 'subcategory', 'task', 'conversation'],
        num_rows: 4298
    })
})

In [9]:
import numpy as np
from collections import defaultdict
fc_num_stat = defaultdict(lambda: 0)

def filter_func(data):
    fcs = json.loads(data["tools"])
    if len(data["conversation"]) >= 10:
        if any(np.array([fc_num_stat[f["function"]["name"]] for f in fcs]) > 200):
            return False
        else:
            for f in fcs:
                fc_num_stat[f["function"]["name"]] += 1
            return True
    elif len(data["conversation"]) > 6:
        if any(np.array([fc_num_stat[f["function"]["name"]] for f in fcs]) > 150):
            return False
        else:
            for f in fcs:
                fc_num_stat[f["function"]["name"]] += 1
            return True
    else:
        if any(np.array([fc_num_stat[f["function"]["name"]] for f in fcs]) > 100):
            return False
        else:
            for f in fcs:
                if fc_num_stat[f["function"]["name"]] < 50:
                    for f in fcs:
                        fc_num_stat[f["function"]["name"]] += 1
                    return True
    return False
    
all_ds["glaive_func_calling"] = all_ds["glaive_func_calling"].filter(filter_func, load_from_cache_file=False)
all_ds

Filter: 100%|██████████| 4298/4298 [00:00<00:00, 13728.16 examples/s]


DatasetDict({
    func_calling_singleturn: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'tools'],
        num_rows: 1783
    })
    func_calling: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'tools'],
        num_rows: 1604
    })
    glaive_func_calling: Dataset({
        features: ['id', 'tools', 'category', 'subcategory', 'task', 'conversation'],
        num_rows: 2810
    })
})

In [12]:
from collections import defaultdict
function_stat = defaultdict(lambda: 0)
for sample in all_ds['glaive_func_calling']:
    if not sample["tools"]:
        function_stat[None] += 1
    else:
        functions = json.loads(sample["tools"]) 
        for f in functions:
            function_stat[f["function"]["name"]] += 1

sorted_stat = sorted(function_stat.items(), key=lambda x: x[1], reverse=True)
print(sorted_stat)
len(sorted_stat)

[('get_stock_price', 201), ('convert_currency', 201), ('generate_random_number', 201), ('calculate_distance', 191), ('get_movie_details', 171), ('search_movies', 153), ('search_recipes', 148), ('get_news', 143), ('calculate_tip', 137), ('calculate_bmi', 134), ('calculate_age', 129), ('calculate_loan_payment', 128), ('search_books', 127), ('calculate_discount', 116), ('translate_text', 104), ('generate_qr_code', 101), ('calculate_mortgage_payment', 99), ('create_todo', 99), ('create_note', 99), ('get_news_headlines', 97), ('search_movie', 87), ('search_restaurants', 86), ('generate_password', 85), ('calculate_area', 81), ('create_calendar_event', 81), ('send_email', 77), ('search_recipe', 71), ('get_definition', 71), ('play_music', 69), ('get_random_joke', 68), ('convert_temperature', 68), ('calculate_tax', 58), ('generate_random_password', 57), ('get_random_fact', 56), ('analyze_sentiment', 55), ('create_user', 44), ('calculate_mortgage', 43), ('calculate_interest', 42), ('search_image

396

In [ ]:
n = 0
for d in all_ds['func_calling_singleturn']:
    for c in d["conversation"]:
        if f := c["function_call"]:
            if len(json.loads(f)) > 1:
                n += 1
                break

n

In [13]:
all_ds.push_to_hub("hqfx/hermes_fc_oai")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.71s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/hqfx/hermes_fc_oai/commit/6998a71617e24b84c9a59084763d6e0a21de7f13', commit_message='Upload dataset', commit_description='', oid='6998a71617e24b84c9a59084763d6e0a21de7f13', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/hqfx/hermes_fc_oai', endpoint='https://huggingface.co', repo_type='dataset', repo_id='hqfx/hermes_fc_oai'), pr_revision=None, pr_num=None)